In [1]:
pip install spotipy

   ---------------------------------------- 0.0/252.0 kB ? eta -:--:--
   ---------------- ----------------------- 102.4/252.0 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 252.0/252.0 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [119]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# change these according to your developer account
client_id = 'look for this on the spotify developer settings'
client_secret = 'look for this on the spotify developer settings'
redirect_uri = 'http://localhost'

# leave the scope and cache the same
scope = 'user-library-read playlist-modify-public playlist-modify-private user-top-read'
cache_path = '.cache-spotify'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope,
                                               cache_path=cache_path,
                                               requests_timeout=30))

In [69]:
def find_top_tracks(sp, limit, time_range):
    top_tracks = sp.current_user_top_tracks(limit=limit, time_range=time_range)['items']
    tracks = []
    artists = []
    for item in top_tracks:
        tracks.append(item['name'])
        artists.append(item['artists'][0]['name'])
    top10_tracks_df = pd.DataFrame({'Artist': artists, 'Track_Name': tracks})
    top10_tracks_df.index = top10_tracks_df.index + 1
    return top10_tracks_df


def find_top_artists(sp, limit, time_range):
    top_artists = sp.current_user_top_artists(limit=limit, time_range=time_range)['items']
    artists = []
    genres = []
    for artist in top_artists:
        artists.append(artist['name'])
        genres.append(", ".join(artist['genres']))   
    top10_artists_df = pd.DataFrame({'Artist': artists, 'Genre': genres})
    top10_artists_df.index = top10_artists_df.index + 1
    return top10_artists_df

In [78]:
# time_range = ['short_term' (4 weeks), 'medium__term' (6 months), 'long_term' (all time)]
find_top_artists(sp, 5, 'long_term')

,Artist,Genre
1,Duki,"argentine hip hop, trap argentino, trap latino..."
2,Eladio Carrion,"trap boricua, trap latino, trap triste, urbano..."
3,Dillom,trap argentino
4,Dire Straits,"album rock, classic rock, mellow gold, rock"
5,Muse,"alternative rock, modern rock, permanent wave,..."


In [110]:
def get_liked_songs(sp, offset=0, limit=50):
    results = sp.current_user_saved_tracks(limit=limit, offset=offset)
    songs = []
    track_ids = []
    
    for item in results['items']:
        track = item['track']
        if track and track['album'] and track['artists']:
            song_info = {
                'Track ID': track['id'],
                'Track Name': track['name'],
                'Artist': track['artists'][0]['name'],
                'Album': track['album']['name'],
                'Date Added': item['added_at'],
                'Artist ID': track['artists'][0]['id'],  # for genre lookup
                'Popularity': track['popularity'],
                'Duration': track['duration_ms'] / (1000 * 60),
                'Album Release Date': track['album']['release_date']
            }
            songs.append(song_info)
            track_ids.append(track['id'])
    
    # audio features
    if track_ids:
        audio_features = sp.audio_features(track_ids)
        for song, features in zip(songs, audio_features):
            if features:
                song['Tempo'] = features['tempo']
                song['Energy'] = features['energy']
                song['Danceability'] = features['danceability']
    return songs


def get_all_liked_songs(sp):
    all_songs = []
    offset = 0
    while True:
        songs = get_liked_songs(sp, offset, limit=50)
        if not songs:
            break
        all_songs.extend(songs)
        offset += 50
    return all_songs


artist_genre = {}

def get_artist_genre(artist_id):
    if artist_id in artist_genre:
        return artist_genre[artist_id]
    artist = sp.artist(artist_id)
    main_genre = artist['genres'][0] if artist['genres'] else 'Unknown'
    artist_genre[artist_id] = main_genre
    return main_genre

In [117]:
genre_mapping = {
    "acid rock": "Rock",
    "acoustic pop": "Pop",
    "adult standards": "Pop",
    "afrofuturism": "Alternative",
    "album rock": "Rock",
    "albuquerque indie": "Indie",
    "alt z": "Alternative",
    "alternative country": "Country",
    "alternative dance": "Dance",
    "alternative hip hop": "Hip Hop",
    "alternative metal": "Metal",
    "alternative r&b": "R&B",
    "alternative rock": "Rock",
    "anime": "Other",
    "argentine alternative rock": "Rock",
    "argentine hip hop": "Hip Hop",
    "argentine rock": "Rock",
    "argentine telepop": "Pop",
    "art pop": "Pop",
    "art rock": "Rock",
    "atl hip hop": "Hip Hop",
    "australian dance": "Dance",
    "australian indie": "Indie",
    "australian indie rock": "Rock",
    "australian pop": "Pop",
    "australian psych": "Rock",
    "australian rock": "Rock",
    "austropop": "Pop",
    "axe": "Latin",
    "bachata": "Latin",
    "bachata dominicana": "Latin",
    "baroque pop": "Pop",
    "beatlesque": "Rock",
    "bedroom pop": "Pop",
    "big room": "Electronic",
    "blues": "Blues",
    "blues rock": "Rock",
    "bolero": "Latin",
    "boy band": "Pop",
    "brazilian reggae": "Reggae",
    "brazilian rock": "Rock",
    "brighton indie": "Indie",
    "brit funk": "Funk",
    "british folk": "Folk",
    "british invasion": "Rock",
    "british power metal": "Metal",
    "british soul": "Soul",
    "britpop": "Rock",
    "britpop revival": "Rock",
    "bronx drill": "Hip Hop",
    "canadian electronic": "Electronic",
    "canadian hip hop": "Hip Hop",
    "canadian pop": "Pop",
    "canadian pop punk": "Pop",
    "canadian singer-songwriter": "Pop",
    "cancion melodica": "Latin",
    "candy pop": "Pop",
    "cantautor": "Latin",
    "celtic rock": "Rock",
    "cha-cha-cha": "Latin",
    "chanson": "Other",
    "charanga": "Latin",
    "chicago rap": "Hip Hop",
    "chicano rap": "Hip Hop",
    "chilean hardcore": "Rock",
    "chilean rock": "Rock",
    "city pop": "Pop",
    "classic italian pop": "Pop",
    "classic rock": "Rock",
    "classic soul": "Soul",
    "classic uk pop": "Pop",
    "classic venezuelan pop": "Pop",
    "classical": "Classical",
    "classical tenor": "Classical",
    "colombian pop": "Pop",
    "complextro": "Electronic",
    "concurso de talentos argentino": "Other",
    "conscious hip hop": "Hip Hop",
    "contemporary country": "Country",
    "corrido": "Latin",
    "corridos tumbados": "Latin",
    "cuatro venezolano": "Latin",
    "cubaton": "Latin",
    "cumbia 420": "Latin",
    "cumbia amazonica": "Latin",
    "cumbia chilena": "Latin",
    "cumbia peruana": "Latin",
    "cumbia pop": "Latin",
    "dance pop": "Pop",
    "dance rock": "Rock",
    "deep new americana": "Americana",
    "deep talent show": "Other",
    "deep tech house": "Electronic",
    "deep tropical house": "Electronic",
    "denpa-kei": "Other",
    "dfw rap": "Hip Hop",
    "disco": "Dance",
    "east coast hip hop": "Hip Hop",
    "east coast reggae": "Reggae",
    "edm": "Electronic",
    "electro": "Electronic",
    "electro latino": "Latin",
    "electronica argentina": "Electronic",
    "emo": "Rock",
    "emo rap": "Hip Hop",
    "epicore": "Other",
    "europop": "Pop",
    "fantasy metal": "Metal",
    "folk-pop": "Folk",
    "french shoegaze": "Alternative",
    "future ambient": "Electronic",
    "g funk": "Hip Hop",
    "garage rock": "Rock",
    "garage rock revival": "Rock",
    "german pop": "Pop",
    "german pop rock": "Rock",
    "girl group": "Pop",
    "glam metal": "Metal",
    "glam rock": "Rock",
    "grunge": "Rock",
    "gruperas inmortales": "Latin",
    "hard rock": "Rock",
    "heartland rock": "Rock",
    "hi-nrg": "Electronic",
    "hip hop": "Hip Hop",
    "hopebeat": "Pop",
    "icelandic indie": "Indie",
    "indie rock": "Rock",
    "indie rock italiano": "Rock",
    "indie rock peruano": "Rock",
    "indietronica": "Indie",
    "instrumental math rock": "Rock",
    "irish rock": "Rock",
    "italian adult pop": "Pop",
    "italian alternative": "Alternative",
    "italian hip hop": "Hip Hop",
    "italian tech house": "Electronic",
    "j-acoustic": "Other",
    "japanese soul": "Soul",
    "japanese teen pop": "Pop",
    "japanese vgm": "Other",
    "j-pop": "Pop",
    "latin afrobeat": "Latin",
    "latin alternative": "Latin",
    "latin arena pop": "Latin",
    "latin hip hop": "Hip Hop",
    "latin house": "Latin",
    "latin jazz": "Jazz",
    "latin pop": "Latin",
    "leicester indie": "Indie",
    "mellow gold": "Other",
    "merengue": "Latin",
    "metropopolis": "Pop",
    "mexican tech house": "Electronic",
    "miami hip hop": "Hip Hop",
    "modern alternative rock": "Rock",
    "modern blues rock": "Rock",
    "modern folk rock": "Rock",
    "modern rock": "Rock",
    "motown": "Soul",
    "movie tunes": "Other",
    "musica chihuahuense": "Latin",
    "native american contemporary": "Other",
    "nederpop": "Pop",
    "neo mellow": "Pop",
    "neo soul": "Soul",
    "new romantic": "Alternative",
    "new wave": "Alternative",
    "new wave pop": "Pop",
    "otacore": "Other",
    "panamanian pop": "Pop",
    "permanent wave": "Alternative",
    "peruvian indie": "Indie",
    "peruvian rock": "Rock",
    "piano rock": "Rock",
    "pop": "Pop",
    "pop argentino": "Pop",
    "pop dance": "Pop",
    "pop peruano": "Pop",
    "pop rap": "Pop",
    "pop reggaeton": "Latin",
    "pop rock": "Rock",
    "pop venezolano": "Pop",
    "pov: indie": "Indie",
    "power pop": "Pop",
    "previa": "Other",
    "puerto rican pop": "Pop",
    "r&b": "R&B",
    "r&b argentino": "R&B",
    "r&b en espanol": "R&B",
    "rap": "Hip Hop",
    "rap canario": "Hip Hop",
    "rap francais": "Hip Hop",
    "rap regio": "Hip Hop",
    "rap underground argentino": "Hip Hop",
    "reggae": "Reggae",
    "reggae fusion": "Reggae",
    "reggaeton": "Latin",
    "reggaeton flow": "Latin",
    "rock drums": "Rock",
    "rock uruguayo": "Rock",
    "rock-and-roll": "Rock",
    "salsa": "Latin",
    "salsa peruana": "Latin",
    "soft rock": "Rock",
    "son cubano": "Latin",
    "southern hip hop": "Hip Hop",
    "spanish indie pop": "Indie",
    "spanish new wave": "Alternative",
    "spanish pop": "Pop",
    "sunshine pop": "Pop",
    "trap argentino": "Hip Hop",
    "trap boricua": "Hip Hop",
    "trap dominicano": "Hip Hop",
    "trap latino": "Hip Hop",
    "Unknown": "Other",
    "vallenato moderno": "Latin",
    "yacht rock": "Rock"
}

In [118]:
liked_songs = get_all_liked_songs(sp)

# Extract and organize data into lists
track_ids = []
track_names = []
artist_names = []
album_names = []
genres = []
date_added = []
popularity = []
duration = []
explicit = []
album_release_dates = []
tempos = []
energies = []
danceabilities = []

for song in liked_songs:
    track_ids.append(song['Track ID'])
    track_names.append(song['Track Name'])
    artist_names.append(song['Artist'])
    album_names.append(song['Album'])
    genres.append(get_artist_genre(song['Artist ID']))  # use function to get genre
    date_added.append(song['Date Added'])
    popularity.append(song['Popularity'])
    duration.append(song['Duration'])
    album_release_dates.append(song['Album Release Date'])
    tempos.append(song.get('Tempo', None))
    energies.append(song.get('Energy', None))
    danceabilities.append(song.get('Danceability', None))

# Create DataFrame
liked_tracks_df = pd.DataFrame({
    'Track ID': track_ids,
    'Track Name': track_names,
    'Artist': artist_names,
    'Album': album_names,
    'Genre': genres,
    'Date Added': date_added,
    'Popularity': popularity,
    'Duration': duration,
    'Album Release Date': album_release_dates,
    'Tempo': tempos,
    'Energy': energies,
    'Danceability': danceabilities
})

liked_tracks_df['Date Added'] = pd.to_datetime(liked_tracks_df['Date Added'])
liked_tracks_df['Date Added'] = liked_tracks_df['Date Added'].dt.tz_convert('America/Lima')
liked_tracks_df['Year Added'] = liked_tracks_df['Date Added'].dt.year
liked_tracks_df['Month Added'] = liked_tracks_df['Date Added'].dt.month
liked_tracks_df['Day Added'] = liked_tracks_df['Date Added'].dt.day
liked_tracks_df['Hour Added'] = liked_tracks_df['Date Added'].dt.hour
liked_tracks_df['Broad Genre'] = liked_tracks_df['Genre'].map(genre_mapping)

liked_tracks_df.to_csv('liked_tracks.csv', index=False)
liked_tracks_df

,Track ID,Track Name,Artist,Album,Genre,Date Added,Popularity,Duration,Album Release Date,Tempo,Energy,Danceability,Year Added,Month Added,Day Added,Hour Added,Broad Genre
0,5rAxhWcgFng3s570sGO2F8,A Place for My Head,Linkin Park,Hybrid Theory (Bonus Edition),alternative metal,2024-06-12 12:45:21-05:00,65,3.077333,2000,133.063,0.908,0.603,2024,6,12,12,Metal
1,0rNqDh9zWWJVTLS4VfceHP,Brickell,Feid,MANIFESTING 20-05,colombian pop,2024-06-11 17:57:50-05:00,79,3.006000,2024-04-10,93.037,0.890,0.805,2024,6,11,17,Pop
2,4kjI1gwQZRKNDkw1nI475M,MY EYES,Travis Scott,UTOPIA,rap,2024-06-10 20:04:56-05:00,79,4.187483,2023-07-28,119.043,0.621,0.455,2024,6,10,20,Hip Hop
3,6F9yAYUaNbUhdlQyt5uZ3b,La Incondicional,Luis Miguel,Busca Una Mujer,bolero,2024-06-07 21:34:11-05:00,73,4.283767,1988-11-25,155.127,0.727,0.374,2024,6,7,21,Latin
4,7alVaT3Dl9jsT1vzcUz6rj,El Día De Mi Suerte,Willie Colón,Greatest Hits,salsa,2024-06-06 09:02:55-05:00,38,5.479767,2008-02-26,97.097,0.855,0.560,2024,6,6,9,Latin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,7Jzsc04YpkRwB1zeyM39wE,R U Mine?,Arctic Monkeys,AM,garage rock,2017-02-13 10:22:27-05:00,0,3.362217,2013-09-10,97.076,0.763,0.509,2017,2,13,10,Rock
1497,4kTd0TND65MUY4BlcmJ2cM,Why'd You Only Call Me When You're High?,Arctic Monkeys,AM,garage rock,2017-02-13 10:20:57-05:00,0,2.685550,2013-09-10,91.989,0.627,0.698,2017,2,13,10,Rock
1498,2x8evxqUlF0eRabbW2JBJd,Fluorescent Adolescent,Arctic Monkeys,Favourite Worst Nightmare,garage rock,2017-02-13 10:16:12-05:00,72,3.064883,2007-04-22,112.056,0.828,0.654,2017,2,13,10,Rock
1499,6nFvbLWccsEydO36fyBBlm,Ruby Tuesday - Stereo Version,The Rolling Stones,Between The Buttons (Remastered),album rock,2017-02-13 10:15:40-05:00,0,3.274883,1967-01-20,104.528,0.543,0.518,2017,2,13,10,Rock


## Functions for creating playlists

In [120]:
# creates a dictionary of the songs depending by genre
def divide_by_genre(liked_tracks_df):
    grouped_tracks = liked_tracks_df.groupby('Broad Genre')
    return grouped_tracks

# created playlists by the name of each genre
def create_playlist(sp, grouped_tracks):
    user_id = sp.current_user()['id']
    for genre, tracks in grouped_tracks:
        playlist_name = str(genre)
        playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description='Created automatically')
        playlist_id = playlist['id']
        # now we move the songs
        track_ids = tracks['Track ID'].tolist()
        for i in range(0, len(track_ids), 100):
            sp.playlist_add_items(playlist_id, track_ids[i:i+100])

In [121]:
liked_tracks_df = pd.read_csv('liked_tracks.csv')
create_playlist(sp, divide_by_genre(liked_tracks_df))